## Imports

In [2]:
import datetime as dt
import yfinance as yf
from pandas_datareader import data as pdr
import pandas as pd
import requests
import bs4 as bs
import pickle
import os
yf.pdr_override()

## Scrapping and saving data (csv)

In [3]:
def save_sp500_tickers():
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text.replace('.', '-')
        ticker = ticker[:-1]
        tickers.append(ticker)
    with open("sp500tickers.pickle", "wb") as f:
        pickle.dump(tickers, f)
    return tickers

def get_data_from_yahoo(reload_sp500=False):    
    if reload_sp500:
        tickers = guardar_sp500_tickers()
    else:
        with open("sp500tickers.pickle", "rb") as f:
            tickers = pickle.load(f)
            if not os.path.exists('financial_data'):
                os.makedirs('financial_data')
    for ticker in tickers:     
                    if not os.path.exists('financial_data/{}.csv'.format(ticker)):
                        tick = yf.Ticker(ticker).info
                        df = pd.DataFrame(list(tick.items())) 
                        df.to_csv('financial_data/{}.csv'.format(ticker))
                        df.rename(columns = {0:'Data', 1:ticker}, inplace=True)
                        print(ticker)
                    else:
                         print('Already have {}'.format(ticker))

save_sp500_tickers()
get_data_from_yahoo()

Already have MMM
Already have ABT
Already have ABBV
Already have ABMD
Already have ACN
Already have ATVI
Already have ADBE
Already have AMD
Already have AAP
Already have AES
Already have AFL
Already have A
Already have APD
Already have AKAM
Already have ALK
Already have ALB
Already have ARE
Already have ALXN
Already have ALGN
Already have ALLE
Already have LNT
Already have ALL
Already have GOOGL
Already have GOOG
Already have MO
Already have AMZN
Already have AMCR
Already have AEE
Already have AAL
Already have AEP
Already have AXP
Already have AIG
Already have AMT
Already have AWK
Already have AMP
Already have ABC
Already have AME
Already have AMGN
Already have APH
Already have ADI
Already have ANSS
Already have ANTM
Already have AON
Already have AOS
Already have APA
Already have AAPL
Already have AMAT
Already have APTV
Already have ADM
Already have ANET
Already have AJG
Already have AIZ
Already have T
Already have ATO
Already have ADSK
Already have ADP
Already have AZO
Already have AV

## Compile data (WiP)

In [6]:
                                                       
def compile_data():
    with open("sp500tickers.pickle", "rb") as f:
        tickers = pickle.load(f)
    
    main_df = pd.DataFrame()

   
    for count, ticker in enumerate(tickers):
        df = pd.read_csv('financial_data/{}.csv'.format(ticker))
        df.set_index("1", inplace=True)
        df.drop('0', 1, inplace=True)
        if main_df.empty:
            main_df = df 
        else:
            main_df = main_df.join(df, how='outer')  
            if count % 10 == 0:
                print(count)
    print(main_df.head())
    main_df.to_csv('compiled_financial_data.csv')


compile_data()



ValueError: columns overlap but no suffix specified: Index(['Unnamed: 0'], dtype='object')